In [ ]:
# default_exp bocos

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nangs.pde import PDE
import numpy as np
from nbdev.showdoc import *

# bocos

> This module contains the different boundary conditions available to work with PDEs

In [ ]:
#export

class Boco():
    "Base class to work with bocos kk"
    def __init__(self):
        self.type = None
        self.bs = None
        self.dataset = None
        self.DataLoader = None
        
    def addBoco(self, input_keys, output_keys):
        "This function is called when a PDE adds this boco"
        print('Override this function to add the required values')

    def check(self, inputs, outputs, params):
        print('Override this function to check everything is ok')

    def summary(self, input_keys, output_keys, param_keys):
        print('Override this function to print the boco summary')

    def setSolverParams(self, bs):
        self.bs = bs

In [ ]:
#export

from nangs.utils import *

class PeriodicBoco(Boco):
    "Periodic boundary condition: The outputs of both inputs will be enforced to be equal."
    def __init__(self, inputs1, inputs2):
        super().__init__()
        self.type = 'periodic'
        
        # check for dict with numpy arrays, same inputs and same length
        checkValidDict(inputs1)
        checkValidDict(inputs2)
        if len(inputs1) != len(inputs2):
            raise Exception('Inputs must have the same length !')
        for k in inputs1:
            if k not in inputs2:
                raise Exception(k + ' must be present in both inputs !')
            if len(inputs1[k]) != len(inputs2[k]):
                raise Exception(k + ' must have same length in both inputs !')
        
        self.inputs1 = inputs1
        self.inputs2 = inputs2
        
    def addBoco(self, input_keys, output_keys):
        "This function is called when a PDE adds this boco"
        
        # check that all inputs are present
        checkDictArray(self.inputs1, input_keys)
        checkDictArray(self.inputs2, input_keys)
        
        # create empty list with same dimensions that inputs in pde
        inputs1_values = [[] for i in input_keys]
        inputs2_values = [[] for i in input_keys]

        # extract arrays from dict and store in list, ordered by inputs in the pde
        for k in self.inputs1:
            ix = input_keys.index(k)
            inputs1_values[ix] = self.inputs1[k]

        for k in self.inputs2:
            ix = input_keys.index(k)
            inputs2_values[ix] = self.inputs2[k]
        
        self.inputs1 = inputs1_values
        self.inputs2 = inputs2_values
            
    def summary(self, input_keys, output_keys, param_keys):
        print('Periodic Boco Summary:')
        print('Input 1: ', {name: values for name, values in zip(input_keys, self.inputs1)})
        print('Input 2: ', {name: values for name, values in zip(input_keys, self.inputs2)})
        print('')

Example

In [ ]:
pde = PDE(inputs=['a', 'b'], outputs=['c'])

a1, a2 = np.array([0]), np.array([1])
b = np.array([1, 2, 3])

boco = PeriodicBoco({'a': a1, 'b': b}, {'a': a2, 'b': b})
pde.addBoco(boco)

pde.bocoSummary()

Periodic Boco Summary:
Input 1:  {'a': array([0]), 'b': array([1, 2, 3])}
Input 2:  {'a': array([1]), 'b': array([1, 2, 3])}



The Periodic boco requires two different sets of inputs. During training, the NN will compute the output of both and enforce them to be equal.

In [ ]:
#hide

pde = PDE(inputs=['a', 'b'], outputs=['c'])

a1, a2 = np.array([0]), np.array([1])
b = np.array([1, 2, 3])

boco = PeriodicBoco({'a': a1, 'b': b}, {'a': a2, 'b': b})
pde.addBoco(boco)

try:
    boco = PeriodicBoco({'a': a1, 'b': b})
except Exception as e:
    assert str(e) == "__init__() missing 1 required positional argument: 'inputs2'", "assertion failed"

try:
    boco = PeriodicBoco({'a': a1, 'b': b}, {'a': a2, 'c': b})
except Exception as e:
    assert str(e) == "b must be present in both inputs !", "assertion failed"
    
try:
    boco = PeriodicBoco({'a': a1, 'c': b}, {'a': a2, 'c': b})
    pde.addBoco(boco)
except Exception as e:
    assert str(e) == "c is not present in ['a', 'b']", "assertion failed"